# traini g using darts library

TL using the same number of IDs is possible

Reshaping does not seem possible.

In [ ]:

from darts import TimeSeries
from darts.utils.losses import *
from darts.metrics import mse
from darts.models import *

In [ ]:
from utils import data_handling, helpers
# use electricity dataset

data_dict = data_handling.format_electricity()

df = data_dict

for key, value in df.items():
			df[key]= data_handling.df_to_tensor(value)


# normalize train and use matrics for val and test
df["train"], train_standardize_dict = helpers.custom_standardizer(df["train"])
df["validation"], _ = helpers.custom_standardizer(df["validation"], train_standardize_dict)
df["test"], _ = helpers.custom_standardizer(df["test"], train_standardize_dict)

ts_train = TimeSeries.from_values(df["train"])
ts_val = TimeSeries.from_values(df["validation"])
ts_test = TimeSeries.from_values(df["test"])

In [ ]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
ts_bavaria_train = TimeSeries.from_values(bavaria_dict["train"])
ts_bavaria_val = TimeSeries.from_values(bavaria_dict["validation"])
ts_bavaria_test = TimeSeries.from_values(bavaria_dict["test"])

In [ ]:
# model definition

# Slicing hyper-params:
IN_LEN = 96
OUT_LEN = 96

# Architecture hyper-params:
NUM_STACKS = 4
NUM_BLOCKS = 1
NUM_LAYERS = 2
LAYER_WIDTH = 128
COEFFS_DIM = 11

# Training settings:
LR = 1e-3
BATCH_SIZE = 64

NUM_EPOCHS = 10

USE_REVIN = True

LOSS_FN = torch.nn.MSELoss()

# reproducibility
np.random.seed(42)
torch.manual_seed(42)

nbeats_model = NBEATSModel(
    input_chunk_length=IN_LEN,
    output_chunk_length=OUT_LEN,
    batch_size=BATCH_SIZE,
    num_stacks=NUM_STACKS,
    num_blocks=NUM_BLOCKS,
    num_layers=NUM_LAYERS,
    layer_widths=LAYER_WIDTH,
    expansion_coefficient_dim=COEFFS_DIM,
    loss_fn=LOSS_FN,
    use_reversible_instance_norm=USE_REVIN,
    optimizer_kwargs={"lr": LR},
    pl_trainer_kwargs={
        "enable_progress_bar": True,
        # change this one to "gpu" if your notebook does run in a GPU environment:
        "accelerator": "gpu",
    },
)


transformer_model = TransformerModel(
    input_chunk_length=IN_LEN, 
    output_chunk_length=OUT_LEN,
    output_chunk_shift=0, 
    d_model=64, 
    nhead=4, 
    num_encoder_layers=3, 
    num_decoder_layers=3, 
    dim_feedforward=512, 
    dropout=0.1, 
    activation='relu', 
    loss_fn=LOSS_FN,
    norm_type=None, 
    custom_encoder=None, 
    custom_decoder=None,
    use_reversible_instance_norm=USE_REVIN,
)

In [ ]:
# Train
nbeats_model.fit(
    ts_train,
    num_loader_workers=4,
    epochs=NUM_EPOCHS,
    max_samples_per_ts=None,
)

preds = nbeats_model.predict(n=96, series=ts_bavaria_test)


In [ ]:
# Train
transformer_model.fit(
    ts_train[:][:66],
    num_loader_workers=4,
    epochs=NUM_EPOCHS,
    max_samples_per_ts=None,
)

preds = transformer_model.predict(n=96, series=ts_bavaria_test[0:96])